In [48]:
import pandas as pd
import numpy as np
import datetime as dt
from datetime import date
import os
import requests
import json
API_TOKEN  = 

In [90]:
# PUT requests to change the sales status of existiong jobs
def update_sales_status(API_TOKEN, uuid, name, status):
    url = f'https://app.followupcrm.com/api/v1/file-data/{uuid}'
    params = {
        'api_token': API_TOKEN,
    }
    payload =  {
        'name': name,
        "external_id": None,
        "external_vendor": None,
        "sales_status": status
    }
    response = requests.put(url, params=params, data = payload)
    if response.status_code == 200:
        json_data = response.json()
        df = pd.json_normalize(json_data['data'])
    else:
        print(f"Error {response.status_code}: {response.text}")

# PUT requests to change the contract date
def update_contract_date(API_TOKEN, uuid, name, new_date):
    url = f'https://app.followupcrm.com/api/v1/file-data/{uuid}'
    params = {
        'api_token': API_TOKEN,
    }
    payload =  {
        'name': name,
        "external_id": None,
        "external_vendor": None,
        'contract_date': new_date
    }
    response = requests.put(url, params=params, data = payload)
    if response.status_code == 200:
        json_data = response.json()
        df = pd.json_normalize(json_data['data'])
    else:
        print(f"Error {response.status_code}: {response.text}")
#creat df from data stored in followup
def get_df(API_TOKEN, page):
    # url = 'https://app.followupcrm.com/api/v1/file-data'
    url = f'https://app.followupcrm.com/api/v1/file-data?page={page}'
    params = {
        'api_token': API_TOKEN,
        'per_page': 15
    }
    response = requests.get(url, params=params)
    if response.status_code == 200:
        json_data = response.json()
        df = pd.json_normalize(json_data['data'])
        # print(len(df))
        return df, json_data
    else:
        print(f"Error {response.status_code}: {response.text}")
# creat and return a subset based on age of the bid and list of sales statuses 
def reduce_dataset(full_df, d_count, status_list):
    full_df['bid_date'] = pd.to_datetime(full_df['bid_date'])
    reduced = full_df[full_df['sales_status'].isin(status_list)]
    reduced = reduced.assign(time_passed = lambda x: pd.Timestamp(date.today()) - x['bid_date'])
    reduced['notify'] = reduced['time_passed'].apply(lambda y: 1 if y > pd.Timedelta(f'{d_count} days 00:00:00') else 0)
    notify = reduced[reduced['notify'] == 1]
    return notify
# subset if specified column is NA 
def missing_value(full_df, field):
    reduced = full_df[full_df[field].isna()]
    return reduced
  

In [80]:
%%time 
df_list = []
json = get_df(API_TOKEN, 1)[1]
last_page = json['meta']['last_page']
for i in range(1,last_page+1):
    df_list.append(get_df(API_TOKEN, i)[0])
# combine dfs
full_df = pd.concat(df_list)

CPU times: total: 391 ms
Wall time: 53.3 s


In [62]:
# subsets to trigger notifications
submits = reduce_dataset(full_df, '14', ["Bidding","New Lead","Submitted"])
budgets = reduce_dataset(full_df, '90', ["Budget"])

In [94]:
no_bid_date = missing_value(full_df, 'bid_date')
no_base_bid = missing_value(full_df, 'base_bid')

In [58]:
# change subset to status update to notify people 
for ind in range(0,len(notify)):
    current = notify.iloc[ind]
    old_status = current['sales_status']
    ## Change the sales status to 
    # update_sales_status(API_TOKEN, current['uuid'], current['name'], 'Budget Loss')
    ## Change the sales status back # this might not work :(
    # update_sales_status(API_TOKEN, current['uuid'], current['name'], old_status)
    print(old_status)

Submitted
Submitted
Submitted
Submitted
Submitted
Submitted
Submitted
Submitted
Submitted
Submitted
Submitted
Submitted
Submitted
Bidding
Bidding
Submitted
Submitted
Submitted
Submitted
Submitted
Submitted
Submitted
Submitted
Submitted
Submitted
Submitted
Bidding
Bidding
Submitted
Submitted
Submitted
Submitted
Submitted
Submitted
Submitted
Submitted
CPU times: total: 15.6 ms
Wall time: 6.02 ms


In [32]:
update_sales_status(API_TOKEN,'b76cb886-bd27-410c-8979-cd3bde1f581a', 'API Test', 'Update Status')

In [31]:
# update_sales_status(API_TOKEN,'be9c5202-e1ed-4c09-9494-b0aae4642ef9', 'Flexential Parker Data Center', 'Update Status')
# update_sales_status(API_TOKEN,'b76cb886-bd27-410c-8979-cd3bde1f581a', 'API Test', 'Lost')

In [30]:
full_df[full_df['name'] == 'Flexential Parker Data Center']["uuid"]

1    be9c5202-e1ed-4c09-9494-b0aae4642ef9
Name: uuid, dtype: object

In [ ]:
for key, value in json.items():
    print(key)

In [9]:
df[df['name'] == 'API Test']['uuid']

463    b76cb886-bd27-410c-8979-cd3bde1f581a
Name: uuid, dtype: object

## Export to Excel

In [14]:
# make path for each day that the data is compiled 
end_file = f'followup_data_{str(date.today())}.xlsx'
end_path = r"C:\Users\will.davis\OneDrive - Fiore & Sons, Inc\1A Data Analysys\P5 Followup API\follow_up_data\{}".format(end_file)
# end_path
# create excel for spot checks
writer = pd.ExcelWriter(end_path, engine="xlsxwriter")
df.to_excel(writer, sheet_name="full_df")
# pivot_table.to_excel(writer, sheet_name="Pivot")
writer.close()

# data set of old budgets to become budget loss

In [ ]:
# data set of old budgets to become budget loss
df2 = df
df2['bid_date'] = pd.to_datetime(df2['bid_date'], format='%m/%d/%Y')
in_range = df2[df2['bid_date'] < pd.to_datetime("2024-12-31")]
# I only want budgets
just_buds = in_range[in_range['sales_status'] == 'Budget']
len(just_buds)
# in_range = df[df['']]
for ind in range(0,len(just_buds)):
    current = just_buds.iloc[ind]
    # print(current['uuid'], current['name'])
    update_sales_status(API_TOKEN, current['uuid'], current['name'], 'Budget Loss')
    print(ind)

# data set of won jobs to update contract date 

In [ ]:
df3 = df
df3['bid_date'] = pd.to_datetime(df3['bid_date'], format='%m/%d/%Y')
df3['dead_date'] = pd.to_datetime(df3['dead_date']).dt.date
# df3['dead_date'] = df3['dead_date'].astype('datetime64[D]')
in_range = df3[df3['bid_date'] < pd.to_datetime("2024-10-15")]
# I only want budgets
just_wins = in_range[in_range['sales_status'] == 'Won']
need_contract_date = just_wins.dropna(subset=['dead_date'])
# need_contract_date = just_wins[just_wins['dead_date'] != pd.NaT]
print(len(just_wins), len(need_contract_date))
for ind in range(0,len(need_contract_date)):
    current = need_contract_date.iloc[ind]
    # print(current['uuid'], current['name'], str(current['dead_date']))
    update_contract_date(API_TOKEN, current['uuid'], current['name'], current['dead_date'])
    print(ind)


In [7]:
df.columns

Index(['id', 'uuid', 'team_id', 'name', 'address', 'city', 'state', 'zip',
       'sales_behaviours', 'account_manager', 'estimator', 'companies',
       'proposal_delivered', 'sales_status', 'office', 'Lead_From',
       'contract_amount', 'base_bid', 'contract_date', 'contract_type',
       'bid_date', 'last_update', 'created_at', 'external_id',
       'external_vendor', 'custom_fields', 'notes', 'outlook_conversation_id',
       'estimated_contract_date', 'gross_margin_percentage', 'labor_hours',
       'dead_date', 'marketing_status', 'pre_bid', 'take_off_person',
       'products_and_services', 'client_profile', 'relate_and_connect',
       'rules_of_engagement', 'find_problems', 'money_step',
       'decision_process', 'change_orders', 'record_detail_custom_fields',
       'contact.name', 'contact.phone', 'contact.fax', 'contact.cell',
       'contact.email'],
      dtype='object')